In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [4]:

def preprocess_image(img):
    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(img, (5, 5), 0)
    # Apply adaptive thresholding to handle uneven lighting
    thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)
    return thresh


In [5]:
def check_symmetry(img, angle, threshold=0.8):
    height, width = img.shape
    center = (width // 2, height // 2)

    # Create rotation matrix
    M = cv2.getRotationMatrix2D(center, angle, 1.0)

    # Rotate image
    rotated = cv2.warpAffine(img, M, (width, height), flags=cv2.INTER_NEAREST)

    # Split the rotated image
    left = rotated[:, :width//2]
    right = rotated[:, width//2:]
    right_flipped = cv2.flip(right, 1)

    # Calculate similarity
    diff = cv2.absdiff(left, right_flipped)
    similarity = 1 - (np.sum(diff) / (width * height * 255))

    return similarity > threshold

In [6]:
def find_symmetries(image_path, angle_step=2, threshold=0.95):
    # Read and preprocess the image
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = preprocess_image(img)

    symmetries = []
    for angle in range(0, 180, angle_step):
        if check_symmetry(img, angle, threshold):
            symmetries.append(angle)

    return symmetries

In [7]:
def plot_symmetries(image_path, symmetries):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    height, width = img.shape
    center = (width // 2, height // 2)

    plt.figure(figsize=(12, 10))
    plt.imshow(img, cmap='gray')

    for angle in symmetries:
        # Calculate end points of the line
        rho = max(width, height)
        a = np.cos(np.radians(angle))
        b = np.sin(np.radians(angle))
        x0 = a * rho + center[0]
        y0 = b * rho + center[1]
        x1 = -a * rho + center[0]
        y1 = -b * rho + center[1]

        plt.plot([x0, x1], [y0, y1], color='r', linestyle='--', label=f'Symmetry at {angle}°')

    plt.title(f'Hand-drawn Image with Detected Symmetry Axes (Total: {len(symmetries)})')
    plt.xlabel('X-axis')
    plt.ylabel('Y-axis')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()

In [ ]:
# Example usage
image_path = '/content/bee6.png'
symmetries = find_symmetries(image_path)
print(f"Number of symmetries found: {len(symmetries)}")
for angle in symmetries:
    print(f"Symmetry found at angle: {angle}°")

plot_symmetries(image_path, symmetries)